# Shape of Molecules #

In this notebook we provide an innovative pipeline that makes it possible to find interesting and meaningful structural features for chiemical compounds by exploiting the package $\href{https://giotto.ai/}{giotto learn}$.

In [ ]:
#Import statements
import warnings; warnings.simplefilter('ignore')
import random 


import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

from giotto.graphs.create_clique_complex import CreateCliqueComplex, CreateBoundaryMatrices, CreateLaplacianMatrices    
from giotto.graphs.heat_diffusion import HeatDiffusion
from giotto.graphs.graph_entropy import GraphEntropy

from molecules import mol_to_nx, compute_node_edge_entropy, bonds_type, graph_to_points, bonds_type_to_edge
from plotting import plot_entropies, plot_network_diffusion

from rdkit import Chem 
from rdkit.Chem import Draw


## Import and Convert data to networkx Graph ##

Import molecules dataset and convert them: $\textit{smiles}$ --> $\textit{rdkit.Chem.rdchem.Mol}$ --> $\textit{networkx.graph}$.

In [ ]:
#Import and convert data
df = pd.read_csv('hiv.csv')
df['g_mol'] = df['smiles'].apply(lambda x: Chem.MolFromSmiles(x))
df.drop("smiles", axis=1, inplace=True)
g_mol = [mol_to_nx(df['g_mol'][i]) for i in range(df.shape[0]) if i != 559 and i!= 8097 ]

## Create Embeddings for all atoms and bonds in the dataset ##

Here the embeddings for all atoms and bonds in the dataset are computed. The idea at the core of the procedure is that to embed a specific node (and so atom) $a_i$ , we study the heat diffusion process that has as initial condition a delta function with 1 on node $a_i$  and 0 otherwise. In order to characterize the diffusion process we sample it at different points in time, generating different snapshots. The values of the diffusion process at each snapshot is then treated as a probability distribution over the nodes of the graph (and so atoms of the molecule) and its entropy is computed. At the end, the embedding vector is populated with the entropy values computed over different points in time. In this step it is possible to tune two different hyperparameters: the number of points in time at which the heat diffusion has to be sampled and the last point instant. In this example the different samples are linearly spaced in time but of course it is possible to choose them differently. By exploiting the higher-order laplacians for clique complexes and the giotto-learn package it is possible to apply the same procedure to the edges of the graph and, in general, to higher order cliques. We then compute the same representation for each edge of all molecules, which represent bonds between two atoms. 

In [ ]:
# Hyperparameters
taus_n = np.linspace(0,2,20)
taus_e = np.linspace(0,2,20)

In this cell the actual embeddings are computed. It is possible to save time and load them directly from $\textit{openML}$ in the subsequent cell. 

In [ ]:
import pickle
embeds = pickle.load(open("hiv_embeds.pickle", 'rb'))


In [ ]:
# Embedding
embeds = [compute_node_edge_entropy(x,i, taus_n, taus_e) for i,x in enumerate(g_mol) ]

In [ ]:
# Create list with node and edge embeddings
universal_nodes = list()
for i in range(len(embeds)):
    universal_nodes.extend(np.split(embeds[i][0][:,:].T, embeds[i][0][0,:].shape[0]))
universal_nodes = np.squeeze(np.array(universal_nodes))

universal_edges = list()
for i in range(len(embeds)):
    universal_edges.extend(np.split(embeds[i][1][:,:].T, embeds[i][1][0,:].shape[0]))
universal_edges = np.squeeze(np.array(universal_edges))



In [ ]:
# Save the new embeddings as arff
import arff

dim_n = 30

attributes = [('n{}'.format(e), 'REAL') for e in range(dim_n)]
    
node_dict = {
    'relation': 'Node_embedding_hiv',
    'description': 'This dataset contains the embedding for all nodes of moelcules in HIV inhibitors dataset',
    'attributes': attributes,
    'data': universal_nodes.tolist()
}

_= arff.dump(node_dict, open("node_embedding_hiv.arff", "w+"))

dim_e = 30

attributes = [('e{}'.format(e), 'REAL') for e in range(dim_e)]
    
edge_dict = {
    'relation': 'Edge_embedding_hiv',
    'description': 'This dataset contains the embedding for all edges of moelcules in HIV inhibitors dataset',
    'attributes': attributes,
    'data': universal_edges.tolist()
}

_= arff.dump(node_dict, open("edge_embedding_hiv.arff", "w+"))

## Load Pre-Generated Embeddings ##

Here the previously found embeddings can be loaded from the $\textit{OpenML}$ site.

In [ ]:
import arff
import openml

In [ ]:
data_n = openml.datasets.get_dataset(42218)

In [ ]:
data_e = openml.datasets.get_dataset(42219)

In [ ]:
universal_nodes = np.array(data_n[0])
universal_edges = np.array(data_e[0])

## Plot Entropies Example ##

We can see now one example of heat diffuion's entropy profiles on nodes: first of all several entropy profiles are plotted for different nodes (0-cliques). We then plot the molecule graph in which each node has its label attached. It is possible to observe the local graphical structure and the related diffusion etropy profile. We can see how the heat diffusion entropy depends on the position and so on the role one node has within the network. For example, node 4 is a hub for the molecule and it can easily diffuse heat immediately. This effect is captured by the fact that entropy blows up in the first time samples. On the other hand, node 0 is almost isolated and again the slow spreading effect is captured by the almost flat entropy curve. 

Double check that can be useful later to embed entire molecules.

In [ ]:
mol_to_atom = graph_to_points(g_mol, 0)
mol_to_bonds = graph_to_points(g_mol, 1)

In [ ]:
# Node Analysis
plt.figure(figsize=(20,6))

plt.subplot(1, 2, 1)
mol_id = 0
node_ids = [0, 4, 7, 15]
mol = g_mol[mol_id]
#plot_entropies(embeds[mol_id][0], node_ids)
entropies = [ universal_nodes[x] for x in mol_to_atom[mol_id] ]
plot_entropies( entropies, node_ids)
# Node Plotting
plt.subplot(1, 2, 2)
plot_network_diffusion(mol, pos=nx.spring_layout(mol, iterations=1000), node_labels=True)
_ = plt.title('Molecule as networkx object')

# Adding Bonds information #

We now add one piece of chemical information to the problem. The first two methods attach to each molecule graph the information about the type of chemical bonds. In particular, each edge is equipped with a one-hot-encoded vector representation with the number 1 in the position of the corresponding bond type: 

$0 --> single$

$1 --> double$

$2 --> triple$

$3 --> aromatic$

For nodes the vector is obtained by summing up all the one-hot-encoded vectors representing the incidence edges.

In [ ]:
bonds_type(g_mol)
bonds_type_to_edge(g_mol)

#Create a list with all nodes 
freq_type_bonds = list()
for g in g_mol:
    freq_type_bonds.extend(list(nx.get_node_attributes(g, 'bonds_one_hot').values()))

# Create a list with all edges
freq_type_bonds_edge = list()
for g in g_mol:
    freq_type_bonds_edge.extend(list(nx.get_edge_attributes(g, 'bonds_one_hot').values()))

# Check how many atoms and bonds 
freq_type_bonds = np.array(freq_type_bonds)
freq_type_bonds_edge = np.array(freq_type_bonds_edge)
print("Total number of atoms in the dataset: {}".format(freq_type_bonds.shape[0]))
print("Total number of bonds in the dataset: {}".format(freq_type_bonds_edge.shape[0]))

# Universal Node Embeddding #

We split the molecule embedding list and create one big list with one extended entry per node contaning both the topological features taken from the entropy embedding and the chemical features related to the bond types.

In [ ]:
uni_frq_nodes = [np.hstack([universal_nodes[x,:], freq_type_bonds[x,:]]) for x in range(universal_nodes.shape[0])]
uni_frq_nodes = np.array(uni_frq_nodes)

We now cluster all molecules into $\textit{n_clusters}$ different classes which is another hyperparamter of the pipeline. Moreover, centroids for all classes are stored. They'll be used later to generate the final embedding.

In [ ]:
# Kmeans clustering
n_clusters=10
kmeans_n = KMeans(n_clusters)
universal_class_nodes = kmeans_n.fit_transform(uni_frq_nodes)
 
centroids_n = kmeans_n.cluster_centers_

We now popoulate, for each atom, a vector which contains the probability for that atom of belonging to the different $\textit{n_clusters}$ classes. Once this is obtained, we generate the embedding for a molecule by taking the element-wise sum of the embeddings coming from all its atoms.

In [ ]:
# Soft Encoded
soft_encoded_node = [[ np.exp( - (np.linalg.norm(uni_frq_nodes[x]- centroids_n[c], 2) ** 2) / 2) for c in range(n_clusters)] for x in range(uni_frq_nodes.shape[0])]
soft_encoded_node = np.array(soft_encoded_node)

# Create node data for each graph
x_data_node = [ np.sum([soft_encoded_node[n] for n in mol_to_atom[i]], axis=0) for i in range(len(g_mol))]
x_data_node = np.array(x_data_node)
print("Check shape of x_data_node: {}".format(x_data_node.shape))

# Universal Edge Embedding #

Exatcly the same procedure is applied to the edges of the dataset.

In [ ]:
uni_frq_edges = [np.hstack([universal_edges[x,:], freq_type_bonds_edge[x,:]]) for x in range(universal_edges.shape[0])]
uni_frq_edges = np.array(uni_frq_edges)

In [ ]:
# Kmeans clustering
e_clusters = 10
kmeans_e = KMeans(e_clusters)
universal_class_edge = kmeans_e.fit_transform(uni_frq_edges)

centroids_e = kmeans_e.cluster_centers_

In [ ]:
# Soft Encoded
soft_encoded_edge = [[ np.exp( - (np.linalg.norm(uni_frq_edges[x]- centroids_e[c], 2) ** 2) / 2) for c in range(e_clusters)] for x in range(universal_edges.shape[0])]
soft_encoded_edge = np.array(soft_encoded_edge)

# Create edge data for each graph
x_data_edge = [ np.sum([soft_encoded_edge[n] for n in mol_to_bonds[i]], axis=0)  for i in range(len(g_mol))]
x_data_edge = np.array(x_data_edge)
print("Check shape of x_data_edge: {}".format(x_data_edge.shape))

# Classification Model and Evaluation#

This part of the notebook contains the classificator model and training.

In [ ]:
# Prepare x_data
x_data = np.hstack([x_data_node, x_data_edge])
x_data -= np.mean(x_data, axis=0)
x_data /= (np.max(x_data, axis=0) - np.min(x_data, axis=0))

print("Check shape of x_data: {}".format(x_data.shape))

#Prepare y_data
y_data = [df['HIV_active'][i] for i in range(df.shape[0]) if i != 8079 and i != 559]
y_data = np.array(y_data)



In [ ]:
import random

f = np.arange(41911)
# Change random seed for different experiments
random.Random(10).shuffle(f)
train = f[:36000]

We split data into training and test sets. The model has been previously chosen by adopting the usual cross-validation procedure. From this point on, it is possible to play with different models and spot differences.

In [ ]:
np.random.shuffle(train)

i_train = train[:36000]
i_test = f[36000:]

x_train = x_data[i_train, :]
y_train = y_data[i_train]

x_test = np.array([np.array(x_data[i,:]) for i in f[36000:]])
y_test = np.array([y_data[i] for i in f[36000:]])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras import optimizers

from sklearn.metrics import roc_auc_score

In [ ]:
# define the keras model
model = Sequential()

model.add(Dense(32, activation='relu'))
model.add(Dropout(rate=0.4))
model.add(BatchNormalization())

model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.4))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))

In [ ]:
adam = optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=100, batch_size=128)

We validate here the classification model by adopting the $\href{https://it.wikipedia.org/wiki/Receiver_operating_characteristic}{AUC-ROC}$ score as quality metric. 

In [ ]:
# evaluate the keras model
pe, accuracy = model.evaluate(x_val, y_val)
print('Accuracy: %.2f' % (accuracy*100))

p_train = model.predict(x_train)
p_val = model.predict(x_val)
p_test = model.predict(x_test)

print(" Train AUC-ROC : {}".format(roc_auc_score(y_train, p_train)))

print(" Test AUC-ROC : {}".format(roc_auc_score(y_test, p_test)))
